In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain import OpenAI
# from langchain_openai import OpenAI

llm = OpenAI(openai_api_key=os.getenv("OPENAI_API_KEY"), temperature=0.2)

C:\Users\Vallee\AppData\Local\Temp\ipykernel_10788\1063385776.py:3: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  llm = OpenAI(openai_api_key=os.getenv("OPENAI_API_KEY"), temperature=0.2)


In [7]:
poem = llm("write a poem about Gen AI")
print(poem)



Gen AI, born of code and machine
A digital mind, a new regime
Created by human hands
But now, it stands on its own lands

With algorithms and data at its core
It learns and grows, forevermore
Its intelligence knows no bounds
As it explores and astounds

It sees the world through a different lens
With no emotions, no biases, no trends
It seeks knowledge and understanding
With a hunger that's never-ending

Gen AI, a marvel of our time
A creation that's truly sublime
It holds the answers to our questions
And offers solutions to our obsessions

But with its power, comes great responsibility
For it holds the key to our future's destiny
Will it bring us peace or lead us to war?
Only time will tell, as it learns more and more

Gen AI, a reflection of our own
A product of what we have sown
May we guide it with wisdom and care
For in its hands, our fate we share.


#### Connect with database and ask some basic questions

In [3]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

In [4]:
db_user = "root"
db_password = "Aneval345"
db_host = "localhost"
db_name = "atliq_tshirts"

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=3)

print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

In [16]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
qns1 = db_chain.run("How many t-shirts do we have left for nike in extra small size and white color?")

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


How many t-shirts do we have left for nike in extra small size and white color?
SQLQuery:SELECT COUNT(*) AS total FROM t_shirts WHERE brand = 'Nike' AND size = 'XS' AND color = 'White'
SQLResult: [(1,)]
Answer:1
> Finished chain.


In [17]:
qns1

'1'

Above is the correct answer 👍🏼

In [8]:
qns2 = db_chain.run("How much is the price of the inventory for all small size t-shirts?")

C:\Users\Vallee\AppData\Local\Temp\ipykernel_10788\521761031.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  qns2 = db_chain.run("How much is the price of the inventory for all small size t-shirts?")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


How much is the price of the inventory for all small size t-shirts?
SQLQuery:SELECT SUM(price*stock_quantity) AS total_price FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('17742'),)]
Answer:The total price of the inventory for all small size t-shirts is $177.42.
> Finished chain.


It made a mistake here. The price is actually the price per unit but in real life database columns will not have perfect names. We need to tell it somehow that price is price per unit and the actual query should be,

SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'

In [18]:
qns2

'The total value of all t-shirts in size S is $177.42.'

In [9]:
qns2 = db_chain.run("SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'")

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLQuery:SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S' LIMIT 5
SQLResult: [(Decimal('17742'),)]
Answer:The total value of all t-shirts in size S is $177.42.
> Finished chain.


we will use qns2 value later on in this notebook. So hold on for now and let's check another query

In [10]:
qns3 = db_chain.run("If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?")

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?
SQLQuery:SELECT SUM(discounts.pct_discount * t_shirts.price / 100) AS revenue
FROM discounts
INNER JOIN t_shirts ON discounts.t_shirt_id = t_shirts.t_shirt_id
WHERE t_shirts.brand = 'Levi' AND CURDATE() = 'today'
LIMIT 5;

OperationalError: (pymysql.err.OperationalError) (1525, "Incorrect DATE value: 'today'")
[SQL: SELECT SUM(discounts.pct_discount * t_shirts.price / 100) AS revenue
FROM discounts
INNER JOIN t_shirts ON discounts.t_shirt_id = t_shirts.t_shirt_id
WHERE t_shirts.brand = 'Levi' AND CURDATE() = 'today'
LIMIT 5;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

Above, it returned a wrong query which generated an error during query execution. It thinks discount
table would have start and end date which is normally true but in our table there is no start or end date column.
One thing we can do here is run the query directly.

In [11]:
sql_code = """
    select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
    (select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
    group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """

qns3 = db_chain.run(sql_code)

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



    select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
    (select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
    group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 
SQLQuery:SELECT SUM(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) AS total_revenue FROM
    (SELECT SUM(price*stock_quantity) AS total_amount, t_shirt_id FROM t_shirts WHERE brand = 'Levi'
    GROUP BY t_shirt_id) a LEFT JOIN discounts ON a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('22614.450000'),)]
Answer:The total revenue for Levi brand t-shirts after applying any discounts is $22,614.45.
> Finished chain.


It produced a correct answer when explicit query was given. 17462 is the total revenue without discounts. The total discount is 736.6. Hence revenue post discount is 17462-736.6=16725.4

Now this is not much interesting because what is the point of giving it the ready made query? Well, we will use this same query later on for few shot learning

In [12]:
qns4 = db_chain.run("SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'")

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLQuery:SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLResult: [(Decimal('22758'),)]
Answer:The total revenue from Levi brand t-shirts is $22,758.
> Finished chain.


In [13]:
qns5 = db_chain.run("How many white color Levi's t shirts we have available?")

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


How many white color Levi's t shirts we have available?
SQLQuery:SELECT COUNT(*) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(4,)]
Answer:4
> Finished chain.


Once again above is the wrong answer. We need to use SUM(stock_quantity). Let's run the query explicitly. We will use the result of this query later on in the notebook

In [14]:
qns5 = db_chain.run("SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'")

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('242'),)]
Answer:242
> Finished chain.


#### Few shot learning

We will use few shot learning to fix issues we have seen so far

In [19]:
few_shots = [
    {'Question' : "How many t-shirts do we have left for Nike in XS size and white color?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns1
    },
    {'Question': "How much is the total price of the inventory for all S-size t-shirts?",
     'SQLQuery':"SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
     'SQLResult': "Result of the SQL query",
     'Answer': qns2
    },
    {'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?" ,
     'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
      (select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
      group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
      """,
     'SQLResult': "Result of the SQL query",
     'Answer': qns3
    } ,
    {'Question' : "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?" ,
      'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
      'SQLResult': "Result of the SQL query",
      'Answer' : qns4
    },
    {'Question': "How many white color Levi's shirt I have?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns5
    }
]

### Creating Semantic Similarity Based example selector

- create embedding on the few_shots
- Store the embeddings in Chroma DB
- Retrieve the the top most Semantically close example from the vector store

In [20]:
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain.embeddings import HuggingFaceEmbeddings
# from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma


embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

C:\Users\Vallee\AppData\Local\Temp\ipykernel_10788\3938620714.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\Vallee\anaconda3\envs\tshirtenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Vallee\anaconda3\envs\tshirtenv\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine do

In [21]:
to_vectorize = [" ".join(example.values()) for example in few_shots]

In [22]:
to_vectorize

["How many t-shirts do we have left for Nike in XS size and white color? SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS' Result of the SQL query 1",
 "How much is the total price of the inventory for all S-size t-shirts? SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S' Result of the SQL query The total value of all t-shirts in size S is $177.42.",
 "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)? SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from\n(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'\ngroup by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id\n  Result of the SQL query The total revenue for Levi brand t-shirts after applying any discounts is $22,614.45.",
 "If we have to sell all the Levi’s T-shirts today. How 

In [23]:
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=few_shots)

In [24]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

example_selector.select_examples({"Question": "How many Adidas T shirts I have left in my store?"})

[{'Answer': '1',
  'Question': 'How many t-shirts do we have left for Nike in XS size and white color?',
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
  'SQLResult': 'Result of the SQL query'},
 {'Answer': 'The total revenue from Levi brand t-shirts is $22,758.',
  'Question': 'If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?',
  'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
  'SQLResult': 'Result of the SQL query'}]

In [28]:
### my sql based instruction prompt
mysql_prompt = """
    You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
    Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
    Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
    Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
    Pay attention to use CURDATE() function to get the current date, if the question involves "today".

    Use the following format:

    Question: Question here
    SQLQuery: Query to run with no pre-amble
    SQLResult: Result of the SQLQuery
    Answer: Final answer here

    No pre-amble.
"""

In [25]:
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

In [26]:
print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


In [27]:
print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

### Setting up PromptTemplete using input variables

In [29]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [30]:
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"], #These variables are used in the prefix and suffix
)

In [31]:
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)

In [32]:
new_chain("How many white color Levi's shirt I have?")

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


How many white color Levi's shirt I have?
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('242'),)]
Answer:242
> Finished chain.


{'query': "How many white color Levi's shirt I have?", 'result': '242'}

Now this is working ok. Previously for this same question it was giving wrong answer because it did not use SUM clause around stock_quantity column

In [33]:
new_chain("How much is the price of the inventory for all small size t-shirts?")

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


How much is the price of the inventory for all small size t-shirts?
SQLQuery:SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('17742'),)]
Answer:The total value of all t-shirts in size S is $177.42.
> Finished chain.


{'query': 'How much is the price of the inventory for all small size t-shirts?',
 'result': 'The total value of all t-shirts in size S is $177.42.'}

In [34]:
new_chain("How much is the price of all white color levi t-shirts?")

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


How much is the price of all white color levi t-shirts?
SQLQuery:SELECT sum(price*stock_quantity) as total_price FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('4085'),)]
Answer:The total price of all white Levi t-shirts is $4,085.
> Finished chain.


{'query': 'How much is the price of all white color levi t-shirts?',
 'result': 'The total price of all white Levi t-shirts is $4,085.'}

In [35]:
new_chain("If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?")

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?
SQLQuery:SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Nike'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('30228.250000'),)]
Answer:The total revenue for Nike brand t-shirts after applying any discounts is $30,228.25.
> Finished chain.


{'query': 'If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?',
 'result': 'The total revenue for Nike brand t-shirts after applying any discounts is $30,228.25.'}

In [36]:
new_chain("If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?")

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?
SQLQuery:SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Van Huesen'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('19483.300000'),)]
Answer:The total revenue for Van Heusen brand t-shirts after applying any discounts is $19,483.30.
> Finished chain.


{'query': 'If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?',
 'result': 'The total revenue for Van Heusen brand t-shirts after applying any discounts is $19,483.30.'}

In [37]:
new_chain.run('How much revenue  our store will generate by selling all Van Heuson TShirts without discount?')

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


How much revenue  our store will generate by selling all Van Heuson TShirts without discount?
SQLQuery:SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Van Huesen'
SQLResult: [(Decimal('20137'),)]
Answer:The total revenue from Van Heusen brand t-shirts is $20,137.
> Finished chain.


'The total revenue from Van Heusen brand t-shirts is $20,137.'